In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen as u_req 
import re
from selenium import webdriver
import pandas as pd

In [2]:
driver = webdriver.Chrome("~/chromedriver")

# Code to extract reviews from one course
---------------------------------------------------------------------------

In [109]:
class Course:
    def __init__(self, name=None, score=None, total_reviews=None, total_ratings=None, description=None, institution=None, url=None):
        self.name = name
        self.score = score
        self.description = description
        self.institution = institution
        self.url = url
        self.total_ratings = total_ratings
        
        # Total reviews that the course has
        self.total_reviews = total_reviews  
        # {Number of stars: List of all reviews with those stars}
        self.reviews = dict()
        # Number of reviews we have
        self.num_reviews = 0  
    
    def add_reviews(self, reviews):
        """Takes in a list of reviews and updates the review dictionary for the course"""
        
        for review in reviews:
            if str(review.stars) not in self.reviews:
                self.reviews[str(review.stars)] = [review] 
            else:
                self.reviews[str(review.stars)].append(review)

            self.num_reviews += 1
    
    def get_info(self):
        return self.__str__()
    
    def __str__(self):
        course_string = f"Name: {self.name} \
        \nScore: {self.score} \
        \nInstitution: {self.institution} \
        \nDescription: '{self.description}' \
        \nTotal Ratings: {self.total_ratings} \
        \nTotal Reviews: {self.total_reviews} \
        \nNumber of Reviews Extracted: {self.num_reviews} \n"
        return course_string
    
    def build_df(self):
        pass
    
    def get_attrs(self):
        pass
    
    def get_reviews(self, num_pages=20):
        pass
    
    def export_df(self):
        pass
    
class Review:
    def __init__(self, stars, text, date):
        self.stars = stars
        self.text = text
        self.date = date
    
    def __str__(self):
        rev_string = f"Number of Stars: {self.stars} \
        \nDate of Review: {self.date} \
        \nText of Review: '{self.text}'\n"
        return rev_string

## Selenium test code
__________________

In [88]:
driver.get("https://www.coursera.org/learn/machine-learning/reviews")

In [89]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [102]:
# DOESN'T WORK FOR NOW -> Attempts of clicking on the filter button to get reviews from different stars
def click_stars(num_stars=2):
    """Given a number of stars, click the drop down button that gets all the stars"""
    try:
        # Drop down menu -> I simply copied xpath from chrome, I hope its right for every course page lol
        menu_path = "//*[@id='root']/div[1]/div/div[4]/div[1]/div[1]/div[3]/button"
        driver.find_element_by_xpath(menu_path).click()
        
        # 2 stars
        star_path = "/html/body/div[14]/div/div/div/div/ul/li[3]"
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, star_path)))
        driver.find_element_by_xpath(star_path).click()
        
    except Exception as e:
        print("Exception is", e)

# THIS WORKS :) -> Get reviews through greedily iterating each page of results without filter
def click_next_page(page_num):
    """Click the next page of results, return resulting html"""
    button_path = "//*[@id='root']/div[1]/div/div[4]/nav/ul/li[13]/button"
    driver.find_element_by_xpath(button_path).click()
    print(f"Loading page {page_num}...")
    
    try:
        WebDriverWait(driver, 45).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='root']/div[1]/div/div[4]/div[2]/p/span"))) # Wait until the results load, this should be adjusted depending on the internet speed
        print("Got it!")
        print()
        
    except:
        print(f"Result page {page_num} did not load, moving on to the next one...")
    
    return driver.page_source


In [101]:
all_reviews = []

def extract_review(page_link):
    soup = BeautifulSoup(page_link, "lxml")
    
    # Get the wrapper of all the the reviews from that page
    results = list(soup.find_all("div", attrs={"class":"review"}))
    
    # Iterate through results in one page and create list of review objects 
    review_list = []

    for result in results:
        rev_wrap = result.contents[0]
        rev_text = rev_wrap.find_all("div", attrs={"class":"reviewText"})[0].text
        rev_date = rev_wrap.find_all("p", attrs={"class":"dateOfReview"})[0].text

        rev_stars_wrap = rev_wrap.find_all("label")
        rev_stars = 0
        for tag in rev_stars_wrap:
            if "Filled Star" in tag.text:
                rev_stars += 1

        review_list.append(Review(rev_stars, rev_text, rev_date))
    
    return review_list


In [103]:
# Get the first page
all_reviews.extend(extract_review(driver.page_source))
pnum = 1

#Get all the rest
for i in range(10):
    pnum += 1
    all_reviews.extend(extract_review(click_next_page(pnum)))

Loading page 2...
Got it!

Loading page 3...
Got it!

Loading page 4...
Got it!

Loading page 5...
Got it!

Loading page 6...
Got it!

Loading page 7...
Got it!

Loading page 8...
Got it!

Loading page 9...
Got it!

Loading page 10...
Got it!

Loading page 11...
Got it!



In [106]:
print(len(all_reviews))
for i in range(20):
    print(all_reviews[i])

110
Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'Enlightening.'

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'Excellent materials & well taught!'

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'I learn a lot from this awesome course by Andrew Ng. The content is very clear delivered. I have no prior knowledge about machine learning and I  still manage to understand and learn from the course.'

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'The intuitions and insights behind the theory provided by Andrew are the most help part of this course.'

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: '很好'

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'Excellent course. Well taught '

Number of Stars: 5         
Date of Review: Jan 15, 2019         
Text of Review: 'Course was well 

In [107]:
# Get other attributes of course 
def get_other_attrs(page_link):
    #driver.get(page_link)
    #Move all attributes here
    pass

about = driver.find_element_by_xpath("//*[@id='root']/div[1]/div/div[2]/div/div/span[1]/span[1]").text
score = driver.find_element_by_xpath("//*[@id='root']/div[1]/div/div[1]/div/div[2]/span").text
name_inst = driver.find_element_by_xpath("//*[@id='root']/div[1]/div/div[1]/div/h2").text.split(",")
name, inst = name_inst[0], name_inst[1]
total_revs = driver.find_element_by_xpath("//*[@id='root']/div[1]/div/div[1]/div/div[2]/div[3]/div[2]/span").text.split(" ")[0]
total_ratings = driver.find_element_by_xpath("//*[@id='root']/div[1]/div/div[1]/div/div[2]/div[2]/span").text.split(" ")[0]

master_ng_course = Course(name=name, score=score, description=about, institution=inst, total_ratings=total_ratings, total_reviews=total_revs)

for arev in all_reviews:
    master_ng_course.add_review(arev)

In [108]:
print(master_ng_course)

Name: Machine Learning         
Score: 4.9         
Institution:  Stanford University         
Description: 'Machine learning is the science of getting computers to act without being explicitly programmed. In the past decade, machine learning has given us
self-driving cars, practical speech recognition, effective web search, and a vastly improved understanding of the human genome. Machine learning is so
pervasive today that you probably use it dozens of times a day without knowing it. Many researchers also think it is the best way to make progress
towards human-level AI. In this class, you will learn about the most effective machine learning techniques, and gain practice implementing them and
getting them to work for yourself. More importantly, you'll learn about not only the theoretical underpinnings of learning, but also gain the ...
VIEW ALL'         
Total Ratings: 90,450         
Total Reviews: 23,081         
Number of Reviews Extracted: 110 



## Trials with BeautifulSoup only
______

In [3]:
course = "https://www.coursera.org/learn/machine-learning/reviews"
client = u_req(course)
page = soup(client.read(), "html.parser")
client.close()

In [4]:
# Get all the the reviews from one page
results = list(page.find_all("div", attrs={"class":"review"}))

In [74]:
# Iterate through results in one page and create list of review objects 
review_list = []

for result in results:
    rev_wrap = result.contents[0]
    rev_text = rev_wrap.find_all("div", attrs={"class":"reviewText"})[0].text
    rev_date = rev_wrap.find_all("p", attrs={"class":"dateOfReview"})[0].text

    rev_stars_wrap = rev_wrap.find_all("label")
    rev_stars = 0
    for tag in rev_stars_wrap:
        if "Filled Star" in tag.text:
            rev_stars += 1
            
    review_list.append(Review(rev_stars, rev_text, rev_date))

In [79]:
review_list[3].stars

5

In [54]:
# Trials with one review
first_review = results[0].contents[0]

# <div class=reviewText_bunchofshit><div class=reviewText> Text here </div></div> 
first_text = list(first_review)[1].text # alternatively, first_review.find_all("div", attrs={"class":"reviewText"})[0].text

first_date = first_review.find_all("p", attrs={"class":"dateOfReview"})[0].text

# <div class=reviewText_bunchofshit><div class=Box_abunchofshit>
# <div class=Star_abunchofmoreshit> <5 label tags representing each star, each has either "Filled Star" or just "Star"> 
# </div></div></div> 
first_stars_wrapper = list(list(first_review)[0]) # alternatively, first_review.find_all("label")
first_stars = 0

for tag in first_stars_wrapper[0]:
    if "Filled Star" in tag.text:
        first_stars += 1
        
first_obj_review = Review(first_stars, first_text, first_date)

In [73]:
first_review.find_all("label")



5

# Trials to get multiple courses automatically
---------------------------------------------------------------------------------------------------

In [155]:
url = "https://www.coursera.org/browse/data-science?facets=skillNameMultiTag%2CjobTitleMultiTag%2CdifficultyLevelTag%2Clanguages%2CentityTypeTag%3ACOURSE%2CpartnerMultiTag%2CcategoryMultiTag%3Adata-science%2CsubcategoryMultiTag&sortField="
client = u_req(url)

page = soup(client.read(), "html.parser")
page = page
client.close()


In [62]:
# Find all results from the query
results = page.find_all("section") 

In [159]:
# i = 0
# 
# # Section wrapper <section class="rc-DomainPage">
# res = list(results[0])
# 
# # First div wrapper <div class="product-offerings-wrapper">
# res_first_div = res[2]
# 
# # Second section wrapper <section class="rc-ProductOfferings">
# res_sect = res_first_div.section
# 
# # Second div wrapper <div class="offerings-wrapper bt3-container body-container">
# # just_results = list(res_sect[1])
# 
# res_sect1 = list(res_sect)[1]
# 
# bro = list(res_sect1.descendants)
# 
# print(res_sect1.find_all("a"))

results = list(page.find_all())

# for tag in page.find_all(True):
#     if "Computer Science" in tag:
#         print(tag)
#     # if tag.name == "a":
#     #     print(tag)
#     #     print()

#print(len(list(res1.descendants)), len(list(res1.children)), len(list(res1.contents)))
# new_results = list(res1.descendants)
# 
# for res in just_results:
#     print("Result number", i)
#     print(res)
#     print()
#     i += 1

<span data-reactid="48">Computer Science</span>
